# ET MODIS - extraer
_Autor:_    __Jesús Casado__ <br> _Revisión:_ __1/1/2021__ <br>

__Introducción__<br>
Datos de MODIS sobre cobertura de nieve en la cuenca del Deva. 
1. Se cargan los datos originales (8 días) y se agregan a diversas frecuencias temporales (mensual y anual). Los datos se muestran en mapas y series.
2. Introducción al análisis de EOFs sobre los datos anteriores.

__Cosas que arreglar__ <br>
Cómo leer atributos del archivo _hdf_ de MODIS.

***

__Índice__ <br>
__[1 Cartografía](#1-Cartografía)__<br>

__[2 Datos MODIS: Terra y Aqua](#2-Datos-MODIS:-Terra-y-Aqua)__<br>
[2.1 Cargar datos brutos](#2.1-Cargar-datos-brutos)<br>
[2.2 Rellenar mapas faltantes](#2.2-Rellenar-mapas-faltantes)<br>
[2.3 Exportar como netCDF](#2.3-Exportar-como-netCDF)<br>

In [1]:
import os
rutaBase = os.getcwd().replace('\\', '/') + '/'

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()
%matplotlib inline
import seaborn as sns
sns.set_style("whitegrid")
from mpl_toolkits.axes_grid1 import make_axes_locatable
from matplotlib.gridspec import GridSpec
from matplotlib import cm
from matplotlib.colors import ListedColormap, BoundaryNorm, LinearSegmentedColormap
from netCDF4 import Dataset, num2date, date2num
from datetime import datetime
from calendar import monthrange
import time
import geopandas as gpd
import matplotlib.patheffects as pe
import matplotlib.animation as animation
from IPython.display import HTML
from pyproj import Proj, transform, CRS
#os.environ['PROJ_LIB'] = r'C:\Anaconda3\pkgs\proj4-4.9.3-vc14_5\Library\share'

In [3]:
os.chdir(rutaBase + '../../../MODIS/py/')
from funciones_MODIS import *
#from funciones_EOF import *
os.chdir(rutaBase)

In [4]:
os.chdir(rutaBase + '../../py')
#from class_MODIS import *
from funciones_raster import *

## 1 Cartografía

In [5]:
# ruta cartografía
#rutaGIS = 'C:/Users/jcr693/OneDrive - UNICAN - Estudiantes/Proyectos/IAHR2020/GIS/'
rutaGIS = os.path.join(rutaBase, '../../data/GIS/')

# importar polígonos de las cuencas
cuenca = gpd.read_file(rutaGIS + "mask_465_deva.shp")

# importar mdt de la cuenca
DEM = read_ascii(rutaGIS + 'dem_465_deva.asc', crs=25830)
DEM.variable = 'elevación'
DEM.units = 'm.s.n.m'

In [9]:
# definir extensión a extraer de MODIS
left, bottom, right, top = cuenca.to_crs(sinusoidal).bounds.loc[0,:]
cellsize = DEM.cellsize
extent = [left - cellsize, right + cellsize, bottom - cellsize, top + cellsize]

## 2 Datos MODIS: Terra y Aqua

### 2.1 Cargar datos brutos

In [7]:
# rutas de entrada y salida
rutaMODIS = 'C:/Users/jcr693/OneDrive - UNICAN - Estudiantes/Cartografia/MODIS/'

In [8]:
# parámetros de la extracción de datos de MODIS
products = {'Terra': 'MOD16A2', 'Aqua': 'MYD16A2'}
var = 'ET_500m'
units = 'm^2/m^2'
description = 'Serie temporal de mapas de ET de la cuenca del Deva obtenidos a partir de MODIS'
dateslim = None#('2013-10-01', '2015-09-30')#None
tiles = ['h17v04']

In [10]:
ET = {}

for sat in products:

    print(sat.upper())
    print('-' * len(sat))
    product = products[sat]

    # extraer datos de MODIS
    modis = MODIS_extract(rutaMODIS, product, var, tiles, dateslim=dateslim,
                          extent=extent, verbose=True)

    # reproyectar a UTM-30N según el vecino más cercano
    modis.reproyectar(DEM.crs, cellsize, n_neighbors=1, weights='distance', p=1, fillna=-999,
                               snap=DEM, inplace=True)
    
    # recortar a la cuenca
    modis.recortar(cuenca, buffer=10, inplace=True)
    
    # eliminar datos negativos
    modis.data[modis.data < 0] = np.nan

    # definir unidades y variable
    modis.units = units
    modis.variable = var.split('_')[0]

    # ruta y archivo donde guardar los datos extraios
    rutaExport = os.path.join(rutaBase, '../data/')
    if os.path.exists(rutaExport) == False:
        os.makedirs(rutaExport)
    filename = '{0}_{1}_Deva.nc'.format(sat, var)
    # exportar los datos como netCDF
    MODIS2netCDF(rutaExport + filename, modis, description)

    # guardar datos en un diccionario
    ET[sat] = modis

    print('\n')    

TERRA
-----
Seleccionar archivos
nº de archivos (fechas): 863

Generar atributos globales
dimensión:		(2400, 2400)
esquina inf. izqda.:	(-1111950.52, 4447802.08)
esquina sup. dcha.:	(      0.00, 5559752.60)

Crear máscaras
dimensión:		(  48,   49)
esquina inf. izqda.:	(-394443.47, 4782916.80)
esquina sup. dcha.:	(-372195.19, 4804701.58)

Importar datos
Fecha 863 de 863: 2019-11-01	||	Tile  1 de  1: h17v04
Paso 863 de 863:	2019-11-01

AQUA
----
Seleccionar archivos
nº de archivos (fechas): 797

Generar atributos globales
dimensión:		(2400, 2400)
esquina inf. izqda.:	(-1111950.52, 4447802.08)
esquina sup. dcha.:	(      0.00, 5559752.60)

Crear máscaras
dimensión:		(  48,   49)
esquina inf. izqda.:	(-394443.47, 4782916.80)
esquina sup. dcha.:	(-372195.19, 4804701.58)

Importar datos
Fecha 797 de 797: 2019-11-01	||	Tile  1 de  1: h17v04
Paso 797 de 797:	2019-11-01



### 2.2 Rellenar mapas faltantes

In [11]:
# rellenar mapas que falten con datos del otro satélite
Terra_, Aqua_ = missingMaps(ET['Terra'], ET['Aqua'])

# guardar en el diccionario
ET = {'Aqua': Aqua_,
      'Terra': Terra_}

NameError: name 'copy' is not defined

In [ ]:
t = -40

fig, axes = plt.subplots(ncols=2, figsize=(10, 5))
for ax, sat in zip(axes, ET):
    ET[sat].plot(t, ax=ax)
    cuenca.plot(color='lightgray', alpha=.7, ax=ax, zorder=0)
    cuenca.boundary.plot(color='k', ax=ax)
    ax.set_title(sat, fontsize=13);

### 2.3 Exportar como netCDF

In [ ]:
for sat in ET:

    # ruta y archivo donde guardar los datos extraios
    rutaExport = os.path.join(rutaBase, '../data/')
    if os.path.exists(rutaExport) == False:
        os.makedirs(rutaExport)
    filename = '{0}_{1}_Deva_rellenado.nc'.format(sat, var)
    
    # exportar los datos como netCDF
    MODIS2netCDF(rutaExport + filename, ET[sat], description)